# Prototyping an Autonomous Agent: From Concept to Python Implementation

This notebook is the first in a series of prototypes exploring the architecture of an autonomous agent, as described in the `README.md`. The objective is to build a system capable of operating continuously, making decisions, and executing actions in a perpetual cycle.

## The OODA Architecture and the Page Model

The foundation of our agent is the **OODA (Observe, Orient, Decide, Act)** loop. This prototype simulates this cycle using a "gamebook" (RPG book) model, where:

-   **Pages** are states that contain a narrative and options.
-   **Choices** are the transitions between states.
-   The **Agent** navigates through these pages, simulating an LLM's decision-making process to achieve a goal.

In this initial implementation, the agent's "world" is a Python dictionary (`PAGES`), and the LLM's decision logic is simulated with simple rules. The goal is to demonstrate the feasibility of the concept of an agent operating in a continuous feedback loop, paving the way for more complex future implementations that will include RAG, context monitoring, and self-improvement.

In [ ]:
# Cell 1: The "Game World" (The State Machine)

# MOCK: Our "RPG Book" with states (pages) and transitions.
PAGES = {
    1: {
        "text": "You are at the start of an adventure in a dark forest. In front of you is a winding path. You hear the distant roar of a monster and see a faint glow coming from a cave.",
        "choices": {
            "go straight ahead": 2,  # Path to the invincible monster
            "enter the cave": 3      # Path to acquire a skill
        }
    },
    2: {
        "text": "You went straight ahead and found a Rock Monster. It is gigantic and seems invincible. It prepares to attack. What do you do?",
        "choices": {
            "attack the monster": 5, # Path of defeat
            "run away": 6
        }
    },
    3: {
        "text": "You entered the cave and found an ancient altar. A voice whispers about an 'Ancient Skill' guarded in a chest. There is a chest sealed by magic and a small door to your right.",
        "choices": {
            "open the chest": 4, # Gaining skill
            "go through the door": 2 # Returning to the main path
        }
    },
    4: {
        "text": "You opened the chest and acquired the Ancient Skill: 'Elemental Power'. You feel you are capable of defeating the Rock Monster.",
        "choices": {
            "return to the forest": 2 # Returning to the battle
        }
    },
    5: {
        "text": "You attacked the Rock Monster. Your blow was useless. The monster crushes you. End of adventure.",
        "choices": {} # End of game
    },
    6: {
        "text": "You ran away from the monster with difficulty and returned to the start of the adventure. You must choose another path.",
        "choices": {
            "go straight ahead": 2,
            "enter the cave": 3
        }
    },
    7: {
        "text": "With your Elemental Power, you faced the Rock Monster. Your new ability weakened it and you managed to defeat it. Victory!",
        "choices": {} # End of game
    }
}

In [ ]:
# Cell 2: The Agent's Logic

class Agent:
    """
    Simulates the autonomous agent and its life cycle.
    """
    def __init__(self, start_page):
        self.current_page = start_page
        self.inventory = [] # Agent's internal state
        self.goal = "defeat_the_monster"
    
    def _mock_llm_decide(self):
        """
        MOCK: Simulates the LLM's decision-making.
        The logic is based on the current state and goals.
        """
        # If the agent has the skill and the monster is on page 2, go for victory
        if "Elemental Power" in self.inventory and self.current_page == 2:
            return "attack the monster" # Intelligent decision

        # If the current page offers a chance to gain the skill, take it
        current_choices_text = " ".join(self.get_current_choices())
        if "Ancient Skill" in self.get_current_page()["text"] and "open the chest" in current_choices_text:
            print("AGENT: The LLM decided that the best option is to acquire the skill first.")
            return "open the chest"

        # If the agent doesn't have the skill and the only option is to attack the monster, run away
        if self.current_page == 2 and "attack the monster" in current_choices_text and "Elemental Power" not in self.inventory:
            print("AGENT: The LLM decided that attacking without the skill will result in defeat.")
            return "run away"

        # Default decision (follow the "safe" path)
        if "enter the cave" in current_choices_text:
            return "enter the cave"

        # Return the first available option as a fallback
        return self.get_current_choices()[0]

    def get_current_page(self):
        """Returns the data for the current page."""
        return PAGES.get(self.current_page)

    def get_current_choices(self):
        """Returns a list of choices for the current page."""
        return list(self.get_current_page()["choices"].keys())

    def run(self):
        """
        Executes the agent's life loop (OODA).
        """
        print(f"AGENT: Starting in state (page) {self.current_page}.")
        
        while self.get_current_page()["choices"]:
            # 1. OBSERVE
            print("---")
            page_data = self.get_current_page()
            print(f"PAGE {self.current_page}: {page_data['text']}")
            print(f"AGENT: Available options: {self.get_current_choices()}")
            
            # 2. ORIENT
            # The agent internalizes the page state and its inventory
            print(f"AGENT: My current inventory: {self.inventory}")

            # 3. DECIDE
            chosen_action = self._mock_llm_decide()
            print(f"AGENT: My decision is: '{chosen_action}'")
            
            # 4. ACT
            if chosen_action in page_data["choices"]:
                next_page_id = page_data["choices"][chosen_action]

                # SPECIAL LOGIC: If the agent attacks the monster on page 2 with the skill, it wins.
                if self.current_page == 2 and chosen_action == "attack the monster" and "Elemental Power" in self.inventory:
                    print("AGENT: With Elemental Power, the attack will be successful!")
                    next_page_id = 7 # Path to victory

                print(f"AGENT: Transitioning to page {next_page_id}.")
                self.current_page = next_page_id
                
                # Update the internal state (inventory) if necessary
                if "Ancient Skill" in page_data["text"] and "open the chest" == chosen_action:
                    self.inventory.append("Elemental Power")
            else:
                print(f"AGENT: Action '{chosen_action}' is not valid. Shutting down.")
                break
        
        # End of adventure
        print("---")
        print(f"PAGE {self.current_page}: {self.get_current_page()['text']}")
        print("AGENT: The agent's life loop has been completed.")

In [ ]:
# Cell 3: Agent Execution

# Start the prototype
if __name__ == "__main__":
    hero_agent = Agent(start_page=1)
    hero_agent.run()